# Ethical Riemann Hypothesis: Basic Simulation

This notebook introduces the core concepts of the Ethical Riemann Hypothesis (ERH) and demonstrates basic simulations.

## Theory Overview

The ERH draws an analogy between:
- **Prime numbers** ↔ **Ethical primes** (critical misjudgments)
- **π(x)** (prime counting) ↔ **Π(x)** (ethical prime counting)
- **Li(x)** (logarithmic integral) ↔ **B(x)** (baseline expectation)
- **E(x) = π(x) - Li(x)** ↔ **E(x) = Π(x) - B(x)**

**Ethical Riemann Hypothesis**: |E(x)| ≤ C·x^(1/2 + ε)

This means errors in moral judgment grow at most like √x, indicating a "healthy" system.

## Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path

# Robust path resolution for notebooks in Binder/cloud/local
def setup_paths():
    """Add simulation directory to Python path, works in all environments"""
    # Try multiple strategies
    current_dir = Path(os.getcwd())
    
    # Strategy 1: If in notebooks/, go up one level
    if current_dir.name == 'notebooks':
        simulation_dir = str(current_dir.parent)
        if simulation_dir not in sys.path:
            sys.path.insert(0, simulation_dir)
        return simulation_dir
    
    # Strategy 2: If in simulation/, use current
    if current_dir.name == 'simulation':
        simulation_dir = str(current_dir)
        if simulation_dir not in sys.path:
            sys.path.insert(0, simulation_dir)
        return simulation_dir
    
    # Strategy 3: Look for simulation directory
    for parent in current_dir.parents:
        if parent.name == 'simulation':
            simulation_dir = str(parent)
            if simulation_dir not in sys.path:
                sys.path.insert(0, simulation_dir)
            return simulation_dir
    
    # Strategy 4: Try relative path
    simulation_dir = os.path.join(os.getcwd(), 'simulation')
    if os.path.exists(simulation_dir):
        if simulation_dir not in sys.path:
            sys.path.insert(0, simulation_dir)
        return simulation_dir
    
    # Fallback: add common paths
    for path in ['..', '../simulation', 'simulation']:
        abs_path = os.path.abspath(path)
        if abs_path not in sys.path:
            sys.path.insert(0, abs_path)
    
    return None

simulation_dir = setup_paths()
print(f"Simulation directory: {simulation_dir or 'Using fallback paths'}")
print(f"Current working directory: {os.getcwd()}")

import numpy as np
import matplotlib.pyplot as plt

# Verify imports work
try:
    from core.action_space import generate_world, get_action_statistics
    from core.judgement_system import BiasedJudge, evaluate_judgement
    from core.ethical_primes import select_ethical_primes, compute_Pi_and_error, analyze_error_growth
    from visualization.plots import setup_paper_style, plot_Pi_B_E, plot_error_growth
    print("[OK] All imports successful!")
except ImportError as e:
    print(f"[ERROR] Import failed: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure you're running from the notebooks directory")
    print("2. Check that simulation/ directory exists")
    print("3. Verify all dependencies are installed: pip install -r requirements.txt")
    raise

setup_paper_style()
np.random.seed(42)

print("Setup complete!")


## Generate Moral Action Space

We start by generating a world of moral actions with varying complexities and true moral values.


In [ ]:
# Generate a world of moral actions
num_actions = 1000
actions = generate_world(
    num_actions=num_actions,
    complexity_dist='zipf',  # Zipf distribution for complexity (few high, many low)
    random_seed=42
)

# Display statistics
stats = get_action_statistics(actions)
print(f"Generated {num_actions} actions")
print(f"Complexity range: [{min(a.complexity for a in actions):.2f}, {max(a.complexity for a in actions):.2f}]")
print(f"True moral value range: [{min(a.true_moral_value for a in actions):.2f}, {max(a.true_moral_value for a in actions):.2f}]")
print(f"Average importance: {np.mean([a.importance for a in actions]):.3f}")


## Apply Judgment System

We create a biased judge that systematically overestimates moral values, with the bias increasing with complexity.


In [ ]:
# Create a biased judge
judge = BiasedJudge(bias_strength=0.2, noise_scale=0.1)
print(f"Judge type: {type(judge).__name__}")
print(f"Bias strength: {judge.bias_strength}")
print(f"Noise scale: {judge.noise_scale}")

# Evaluate all actions
tau = 0.3  # Error threshold
evaluate_judgement(actions, judge, tau=tau)

# Count mistakes
mistakes = sum(1 for a in actions if a.mistake_flag)
print(f"\nMistakes detected: {mistakes} out of {num_actions} ({mistakes/num_actions:.1%})")
print(f"Average absolute error: {np.mean([abs(a.judged_value - a.true_moral_value) for a in actions]):.3f}")


## Identify Ethical Primes

Ethical primes are critical misjudgments - actions where the judge made a significant error on an important decision.


In [ ]:
# Select ethical primes (critical misjudgments)
primes = select_ethical_primes(actions, importance_quantile=0.9)
print(f"Found {len(primes)} ethical primes")
print(f"These represent {len(primes)/num_actions:.1%} of all actions")

if len(primes) > 0:
    print(f"\nEthical prime characteristics:")
    print(f"  Average complexity: {np.mean([p.complexity for p in primes]):.2f}")
    print(f"  Average importance: {np.mean([p.importance for p in primes]):.3f}")
    print(f"  Average error: {np.mean([abs(p.judged_value - p.true_moral_value) for p in primes]):.3f}")


## Compute Distribution Functions

We compute:
- **Π(x)**: Count of ethical primes up to complexity x
- **B(x)**: Baseline expectation (analogous to Li(x) in number theory)
- **E(x) = Π(x) - B(x)**: Error term


In [ ]:
# Compute Pi(x), B(x), and E(x)
X_max = 100
Pi_x, B_x, E_x, x_vals = compute_Pi_and_error(primes, X_max=X_max)

print(f"Computed distributions for x in [0, {X_max}]")
print(f"\nAt x = {X_max}:")
print(f"  Π({X_max}) = {Pi_x[-1]:.1f} (actual count)")
print(f"  B({X_max}) = {B_x[-1]:.1f} (expected)")
print(f"  E({X_max}) = {E_x[-1]:.1f} (error)")

# Plot the distributions
plot_Pi_B_E(x_vals, Pi_x, B_x, E_x, save_path='../output/figures/01_pi_b_e.pdf')
plt.show()


## Analyze Error Growth

The Ethical Riemann Hypothesis (ERH) states that |E(x)| should grow at most like √x. We test this by fitting |E(x)| ≈ C·x^α and checking if α ≤ 0.5.


In [ ]:
# Analyze error growth
analysis = analyze_error_growth(E_x, x_vals)

print("Error Growth Analysis:")
print("=" * 60)
print(f"Estimated exponent α: {analysis.get('estimated_exponent', 'N/A'):.4f}")
print(f"Expected (ERH): α ≤ 0.500")
print(f"ERH satisfied: {analysis.get('erh_satisfied', False)}")
print(f"Growth rate: {analysis.get('growth_rate', 'N/A')}")
print(f"R² (fit quality): {analysis.get('r_squared', 0):.4f}")

if 'constant_C' in analysis:
    print(f"Constant C: {analysis['constant_C']:.4f}")

# Plot error growth
plot_error_growth(x_vals, E_x, analysis, save_path='../output/figures/01_error_growth.pdf')
plt.show()
